In [1]:
using Test, DataFrames, CSV, Printf
using Plots, LightGraphs, StatsBase

include("markov_aux2.jl")
include("markov2.jl")

nᵢ = CSV.read("C:/Users/pierg/Documents/Julia/MMCA-vaccination/population_surface.csv", DataFrame)
sᵢ = Float64.( nᵢ[:, "Superficie"] )
nᵢ = nᵢ[:, "Population"]
N_tot = sum(nᵢ)

G = 3
M = 52
V = 2

Nᵍ= [12009992, 26409793, 8906906] # Data from INE, provisional as of July 1, 2021 

nᵢᵍ =  Nᵍ/sum(Nᵍ) * nᵢ'
nᵢ = sum(nᵢᵍ, dims = 1)
N_tot = sum(nᵢᵍ)

C = [0.5980 0.3849 0.0171
     0.2440 0.7210 0.0350
     0.1919 0.5705 0.2376]

data = CSV.read("C:/Users/pierg/Documents/Julia/MMCA-vaccination/from_to_flow_provinces.csv", DataFrame)
Rᵢⱼ = zeros(M, M)
for count in 1:nrow(data)
    i = data[count,2]
    j = data[count, 3]
    
    Rᵢⱼ[i, j] = data[count, 4]
end
Rᵢⱼ = Rᵢⱼ ./ sum(Rᵢⱼ, dims = 2)

edgelist, Rᵢⱼ = make_edls(Rᵢⱼ)

kᵍ = [11.8, 13.3, 6.6]
kᵍ_h = [3.15, 3.17, 3.28]
kᵍ_w = [1.72, 5.18, 0.0]
pᵍ = [0.3, 1., 0.05] # [0.3, 1.0, 0.05]
# sᵢ = sample([15., 16., 17., 18, 19., 20., 21., 22., 23., 24., 25. ], M)
ξ = 0.01
σ = 2.5;

In [2]:
# FIRST OF ALL RUN THE SIMULATION WITHOUT VACCINATIONS

# include("markov_aux2.jl")
# include("markov2.jl")

βᴵᵥ = 0.056
βᴬᵥ = 0.5 * βᴵᵥ
ηᵍ = [1, 1, 1] * 0.127
αᵍ = [1, 1, 1] * 0.306
μᵍ = [1, 1, 1] * 0.589

# Rows are the age groups, columns are the vaccination stages
θᵍ = [0.     0.0*(1-0.95);
      0.0  0.0*(1-0.95); 
      0.0  0.0*(1-0.95)]
γᵍ = [0.003  0.003*(1-0.9);
      0.01   0.01*(1-0.9); 
      0.08   0.08*(1-0.9)] 
ωᵍ = [0.  0.0*(1-0.95); 
      0.04  0.04*(1-0.95); 
      0.3  0.3*(1-0.95)]

ζᵍ = [1/7.084, 1/7.084, 1/7.084]
λᵍ = [1/4.084, 1/4.084, 1/4.084]
ψᵍ = [1/7.0, 1/7.0, 1/7.0]
χᵍ = [1/20.0, 1/20.0, 1/20.0]

Λ = 1 / 50 # Waning immunity rate 
Γ = 1 / 100 # Reinfection rate

T = 600

ϵᵍ = [0.1 , 0.4 , 0.5] * round( N_tot / 200 )
tᶜs = [400, 400+60, T]
ϵᵍs = ϵᵍ .* [0 0 0] # reshape(ϵᵍ, (3, 1))

κ₀s = ones(3) * 0.
ϕs = ones(3) 
δs = ones(3) * 0.

rᵥ = [0., 0.6 ] # Relative risk reduction of the probability of infection
kᵥ = [0., 0.4 ] # Relative risk reduction of the probability of transmission

if (@isdefined epi_params)
    reset_params!(epi_params, population)
end    
population = Population_Params(G, M, nᵢᵍ, kᵍ, kᵍ_h, kᵍ_w, C, pᵍ, edgelist, Rᵢⱼ, sᵢ, ξ, σ);
epi_params = Epidemic_Params(βᴵᵥ, βᴬᵥ, ηᵍ, αᵍ, μᵍ, θᵍ, γᵍ,
                             ζᵍ, λᵍ, ωᵍ, ψᵍ, χᵍ, Λ, Γ, rᵥ, kᵥ, G, M, T, V);

## Initialization of the epidemics

E₀ = population.nᵢᵍ/ N_tot * 1000
A₀ = population.nᵢᵍ / N_tot * 1000
I₀ = population.nᵢᵍ / N_tot * 1000
H₀ = population.nᵢᵍ * 0
R₀ = population.nᵢᵍ / N_tot * 23e5 ### NUMERO REALISTICO?
S₁ = (population.nᵢᵍ .- E₀ .- A₀ .- I₀ .- H₀ .- R₀) .* 0.5

set_initial_conditions!(epi_params, population, S₁, E₀, A₀, I₀, H₀, R₀)

# run_epidemic_spreading_mmca!(epi_params, population; ϵᵍ, verbose = true)
run_epidemic_spreading_mmca!(epi_params, population, tᶜs, κ₀s, ϕs, δs, ϵᵍs; verbose = false)

In [3]:
reset_params!(epi_params, population)
set_initial_conditions!(epi_params, population, S₁, E₀, A₀, I₀, H₀, R₀)

# run_epidemic_spreading_mmca!(epi_params, population; ϵᵍ, verbose = true)
run_epidemic_spreading_mmca!(epi_params, population, tᶜs, κ₀s, ϕs, δs, ϵᵍs; verbose = false)

In [4]:
# CALL THE SIMULATION WITHOUT VACCINATION "BASELINE"

baseline = (cases = time_series(epi_params, population).cases,
            hosps = time_series(epi_params, population).icus,
            deaths = time_series(epi_params, population).deaths )

CSV.write("baseline_new.csv", DataFrame(baseline))

# We define the second peak as the next-to-highest one
idx = maxima(baseline.cases).height .== sort(maxima(baseline.cases).height, rev=true)[2]
idx = findall(idx)[1]

t_peak = maxima(baseline.cases).position[1]
t_peak2 = maxima(baseline.cases).position[idx]

h_peak_cases2 = maxima(baseline.cases).height[idx]
h_peak_hosp2 = maxima(baseline.hosps).height[idx]

t_range = t_peak2 - t_peak

184

In [81]:
col_names = ["starting_time", "duration", "priority_old", "priority_adult", "cases_2peak", "hosp_2peak", "cases_cum", "hosp_cum", "deaths_cum", "cases_delay", "hosp_delay"]
t_start = round.(Int64, range(1, stop = Int(floor(t_peak/2 + t_peak2/2)) , length = 15 ) )
t_start = (t_start .+ last(t_start) .- 1)[2:7]
V_tot = round( N_tot / 100 ) * 30    # equivalent to 1% of the population vaccinated per day for thirty days
duration = [30, 60, 90, 120]
old_priority = 1:1:8
# old_priority = 2
T = 500

N_simulation = Int( length(t_start) * length(duration) * (length(old_priority)) * (length(old_priority) + 1) / 2 )
N_parameters = 4 # starting time, duration, old and adult priority + relative change
N_col = length(col_names)

# Define the dataframe in which the result of the simulation will go will go
data = ones(N_simulation, N_col)
data = DataFrame(data, col_names)

aux_cases = ones(T);
aux_hosps = ones(T);
aux_deaths = ones(T);
N_simulation

864

In [82]:
count = 1
@printf("Duration  \t Old   Adult   Young \n")

for d in duration
    for old in old_priority
        adult_priority = (old+1):1:9
        for adult in adult_priority
            for t in t_start
                ϵᵍ = [(10 - adult), adult - old, old] * round( V_tot / d ) / 10 # We divided by 10 because we want old and adult to sum up to unity 
                tᶜs = [t, t + d, T]
                ϵᵍs = ϵᵍ .* [0 1 0] # reshape(ϵᵍ, (3, 1))

                if (@isdefined epi_params)
                    reset_params!(epi_params, population)
                end    
                # population = Population_Params(G, M, nᵢᵍ, kᵍ, kᵍ_h, kᵍ_w, C, pᵍ, edgelist, Rᵢⱼ, sᵢ, ξ, σ);
                epi_params = Epidemic_Params(βᴵᵥ, βᴬᵥ, ηᵍ, αᵍ, μᵍ, θᵍ, γᵍ,
                                             ζᵍ, λᵍ, ωᵍ, ψᵍ, χᵍ, Λ, Γ, rᵥ, kᵥ, G, M, T, V);

                set_initial_conditions!(epi_params, population, S₁, E₀, A₀, I₀, H₀, R₀)
                run_epidemic_spreading_mmca!(epi_params, population, tᶜs, κ₀s, ϕs, δs, ϵᵍs; verbose = false) 

                aux_cases .= time_series(epi_params, population).cases
                aux_hosps .= time_series(epi_params, population).icus
                aux_deaths .= time_series(epi_params, population).deaths

                max = maxima(aux_cases).height
                if length(max) == 1
                    @printf("PROBLEM: T too small, no second wave detected")
                end

                if max[1] == maximum(aux_cases)
                    idx_c = max .== sort(max, rev=true)[2]
                else
                    idx_c = max .== sort(max, rev=true)[1]
                end
                idx_c = findall(idx_c)[1]

                max = maxima(aux_hosps).height
                if max[1] == maximum(aux_hosps)
                    idx_h = max .== sort(max, rev=true)[2]
                else
                    idx_h = max .== sort(max, rev=true)[1]
                end
                idx_h = findall(idx_h)[1]

                data[count, "cases_2peak"] = (maxima(aux_cases).height[idx_c] - h_peak_cases2) / h_peak_cases2
                data[count, "hosp_2peak"] = (maxima(aux_hosps).height[idx_h] - h_peak_hosp2) / h_peak_hosp2
                data[count, "cases_cum"] = (cumsum(aux_cases)[T] - cumsum(baseline.cases)[T]) / cumsum(baseline.cases)[T]
                data[count, "hosp_cum"] = (cumsum(aux_hosps)[T] - cumsum(baseline.hosps)[T]) / cumsum(baseline.hosps)[T]
                data[count, "deaths_cum"] = (aux_deaths[T] - baseline.deaths[T]) / baseline.deaths[T]
                data[count, "cases_delay"] = maxima(aux_cases).position[idx_c] - t_peak2
                data[count, "hosp_delay"] = maxima(aux_hosps).position[idx_h] - maxima(baseline.hosps).position[2]
                data[count, "starting_time"] = t
                data[count, "duration"] = d
                data[count, "priority_old"] = old / 10
                data[count, "priority_adult"] = (adult - old) / 10
                
                count = count + 1  
            end
            @printf("%u        \t %.2f  %.2f    %.2f  \t  Simulation %u out of %u  \t Progress %.2f %% \n", d, old / 10, (adult - old) / 10, (10 - adult) / 10, count - 1, N_simulation, (count-1)/N_simulation*100 )
        end
    end
end

Duration  	 Old   Adult   Young 
30        	 0.10  0.10    0.80  	  Simulation 6 out of 864  	 Progress 0.69 % 
30        	 0.10  0.20    0.70  	  Simulation 12 out of 864  	 Progress 1.39 % 
30        	 0.10  0.30    0.60  	  Simulation 18 out of 864  	 Progress 2.08 % 
30        	 0.10  0.40    0.50  	  Simulation 24 out of 864  	 Progress 2.78 % 
30        	 0.10  0.50    0.40  	  Simulation 30 out of 864  	 Progress 3.47 % 
30        	 0.10  0.60    0.30  	  Simulation 36 out of 864  	 Progress 4.17 % 
30        	 0.10  0.70    0.20  	  Simulation 42 out of 864  	 Progress 4.86 % 
30        	 0.10  0.80    0.10  	  Simulation 48 out of 864  	 Progress 5.56 % 
30        	 0.20  0.10    0.70  	  Simulation 54 out of 864  	 Progress 6.25 % 
30        	 0.20  0.20    0.60  	  Simulation 60 out of 864  	 Progress 6.94 % 
30        	 0.20  0.30    0.50  	  Simulation 66 out of 864  	 Progress 7.64 % 
30        	 0.20  0.40    0.40  	  Simulation 72 out of 864  	 Progress 8.33 % 
30      

90        	 0.50  0.30    0.20  	  Simulation 606 out of 864  	 Progress 70.14 % 
90        	 0.50  0.40    0.10  	  Simulation 612 out of 864  	 Progress 70.83 % 
90        	 0.60  0.10    0.30  	  Simulation 618 out of 864  	 Progress 71.53 % 
90        	 0.60  0.20    0.20  	  Simulation 624 out of 864  	 Progress 72.22 % 
90        	 0.60  0.30    0.10  	  Simulation 630 out of 864  	 Progress 72.92 % 
90        	 0.70  0.10    0.20  	  Simulation 636 out of 864  	 Progress 73.61 % 
90        	 0.70  0.20    0.10  	  Simulation 642 out of 864  	 Progress 74.31 % 
90        	 0.80  0.10    0.10  	  Simulation 648 out of 864  	 Progress 75.00 % 
120        	 0.10  0.10    0.80  	  Simulation 654 out of 864  	 Progress 75.69 % 
120        	 0.10  0.20    0.70  	  Simulation 660 out of 864  	 Progress 76.39 % 
120        	 0.10  0.30    0.60  	  Simulation 666 out of 864  	 Progress 77.08 % 
120        	 0.10  0.40    0.50  	  Simulation 672 out of 864  	 Progress 77.78 % 
120        	

In [33]:
old_priority = 1:1:8
j = 1
for old in old_priority
    adult_priority = (old+1):1:9
    for adult in adult_priority
        ϵᵍ = [(10 - adult), adult - old, old]
        @printf("%.2f %.2f %.2f \n", old / 10 , (adult - old)/ 10, (10 - adult)/ 10 )
        j = j + 1
    end
    @printf("---------------------------\n")
end
@printf("%u", j-1)


0.10 0.10 0.80 
0.10 0.20 0.70 
0.10 0.30 0.60 
0.10 0.40 0.50 
0.10 0.50 0.40 
0.10 0.60 0.30 
0.10 0.70 0.20 
0.10 0.80 0.10 
---------------------------
0.20 0.10 0.70 
0.20 0.20 0.60 
0.20 0.30 0.50 
0.20 0.40 0.40 
0.20 0.50 0.30 
0.20 0.60 0.20 
0.20 0.70 0.10 
---------------------------
0.30 0.10 0.60 
0.30 0.20 0.50 
0.30 0.30 0.40 
0.30 0.40 0.30 
0.30 0.50 0.20 
0.30 0.60 0.10 
---------------------------
0.40 0.10 0.50 
0.40 0.20 0.40 
0.40 0.30 0.30 
0.40 0.40 0.20 
0.40 0.50 0.10 
---------------------------
0.50 0.10 0.40 
0.50 0.20 0.30 
0.50 0.30 0.20 
0.50 0.40 0.10 
---------------------------
0.60 0.10 0.30 
0.60 0.20 0.20 
0.60 0.30 0.10 
---------------------------
0.70 0.10 0.20 
0.70 0.20 0.10 
---------------------------
0.80 0.10 0.10 
---------------------------
36